#[Your_name]

<h1 style="text-align: center;">Arabic Dialect Identification<h1>

Prepare libraries

In [1]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import pandas as pd
import pathlib

Load data

In [15]:

# Load the unification DataFrame
dnu_df = pd.read_excel("dialect_name_unification.xlsx", sheet_name="Sheet1", header=[0, 1])

# Set up the root directory for MADAR data
madar_rootdir = pathlib.Path(r'MADAR/MADAR_Corpus')

# Read all TSV files from the MADAR directory and combine them into one DataFrame
madar_data = [pd.read_csv(file, sep='\t') for file in madar_rootdir.glob('*.tsv')]
madar_df = pd.concat(madar_data)

# Load QADI datasets and add split column
qadi_filepaths = {
    'train': "QADI/data/train-00000-of-00001.parquet",
    'test': "QADI/data/test-00000-of-00001.parquet",
    'validation': "QADI/data/validation-00000-of-00001.parquet"
}

qadi_dfs = {
    split: pd.read_parquet(filepath, engine="pyarrow").assign(split=f'{split}')
    for split, filepath in qadi_filepaths.items()
}

qadi_df = pd.concat(qadi_dfs.values())

<h2 dir="rtl">مثال عن كيفية تنظيم حلول الطلبات:

قم بوضع الخلايا الأربع التالية لحل كل طلب</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [ ]:
# your code here


In [ ]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# مهمة 01  : تحميل البيانات

## [1.1]

In [9]:
# Create a mapping for replacement
split_mapping = {
    'corpus-6-train': 'train',
    'corpus-6-test-corpus-26-train': 'train',
    'corpus-6-dev': 'validation',
    'corpus-6-test-corpus-26-dev': 'validation',
    'corpus-6-test-corpus-26-test': 'test'
}

# Replace the 'split' values using the mapping
madar_df['split'] = madar_df['split'].replace(split_mapping)

# MADAR dataframe
print(f"the length of madar data: {len(madar_df)}")
madar_df.head(10)

the length of madar data: 136000


,sentID.BTEC,split,lang,sent
0,5,train,ALE,هو هنيك، قدام معلومات السياح بالضبط.
1,9,train,ALE,مالي سمعان من قبل بهالعنوان هون.
2,11,train,ALE,روح ساوي لبين ما تشوف صيدلية.
3,26,train,ALE,بشقد الفطور؟
4,27,train,ALE,شلون بقدر أساعدك؟
5,30,train,ALE,روح عاليسار عند تالت مفرق.
6,50,train,ALE,بتريد كريمة وسكر بقهوتك؟
7,64,train,ALE,خرج تصرفلي شيك الميتين دولار تبعي؟
8,74,train,ALE,إذا لازم يبين، اتصل فيني إذا سمحت.
9,86,train,ALE,وين الكافيه؟


In [8]:
## QADI dataframe
print(f"the length of qadi data: {len(qadi_df)}")
qadi_df.head(10)

the length of qadi data: 458197


,id,label,text,split
0,1046024946705735552,9,@Ahmed_Hamza27 اتمنى صراحة و خصوصا #طال_عمره ب...,corpus-6-train
1,1140189387508134016,17,@rJiM4CnIFTgml9g @zamnissi طبعا عرفته من اللي ...,corpus-6-train
2,1051416181855440768,0,@OmanisFollowers @ooredoo @TRA_OMAN @motc_om ه...,corpus-6-train
3,781770259422412800,11,كلب شو كنتوا بتتوقعوا منه !؟ \n\nمحمود عباس لا...,corpus-6-train
4,968140551366340608,11,#عزام 🔊\nاحنا ابو الولد وام الولد,corpus-6-train
5,1129830890069270400,2,@HussamAAlsaleh1 الله يبشرررررك بالجنة يابو سع...,corpus-6-train
6,1055146289791258624,4,@Ocxoc @AlArabiya_Brk والله العظيم انه مجرد تق...,corpus-6-train
7,1040998864327794816,17,@anasyounis52 اها مستوى أقل شوية هههههههه,corpus-6-train
8,966103834618744832,10,قلبي مقبوض ، بقالي كام يوم مش متظبط و اتمني ان...,corpus-6-train
9,1126962565261811584,2,@ryanovic_nfc خوينا ذا 😒 مانخلي اخويانا,corpus-6-train


In [10]:
print(f"madar NAN values: \n{madar_df.isna().sum()}", f"\nmadar duplicated values: \n{madar_df.duplicated().sum()}")

madar NAN values: 
sentID.BTEC        0
split              0
lang               0
sent           24000
dtype: int64 
madar duplicated values: 
0


In [11]:
print(f"qadi NaN values:\n{qadi_df.isna().sum()}", f"\nqadi duplicated values:{qadi_df.duplicated().sum()}")

qadi NaN values:
id       0
label    0
text     0
split    0
dtype: int64 
qadi duplicated values:0


## [1.2]

In [14]:
# Create mapping dictionaries from unification_df
columns_to_map = {
    'city': ('City', 'English name'),
    'country': ('Country', 'English name'),
    'region': ('Region', 'English name')
}

for col, (level1, level2) in columns_to_map.items():
    mapping = dict(zip(dnu_df[('City', 'Abbriviation')], dnu_df[(level1, level2)]))
    madar_df[col] = madar_df['lang'].map(mapping)

# Display the first few rows of the MADAR dataframe
madar_df.head()

,sentID.BTEC,split,lang,sent,city,country,region
0,5,corpus-6-test-corpus-26-train,ALE,هو هنيك، قدام معلومات السياح بالضبط.,Aleppo,Syria,Levant
1,9,corpus-6-test-corpus-26-train,ALE,مالي سمعان من قبل بهالعنوان هون.,Aleppo,Syria,Levant
2,11,corpus-6-test-corpus-26-train,ALE,روح ساوي لبين ما تشوف صيدلية.,Aleppo,Syria,Levant
3,26,corpus-6-test-corpus-26-train,ALE,بشقد الفطور؟,Aleppo,Syria,Levant
4,27,corpus-6-test-corpus-26-train,ALE,شلون بقدر أساعدك؟,Aleppo,Syria,Levant


## [1.3]

In [16]:
madar_df = madar_df.rename(columns={"sentID.BTEC": 'id'})
qadi_df = qadi_df.rename(columns={'text': 'sent'})
qadi_df = qadi_df.reindex(columns=madar_df.columns)

data = pd.concat([madar_df, qadi_df])
data.head()

,id,split,lang,sent
0,5,corpus-6-test-corpus-26-train,ALE,هو هنيك، قدام معلومات السياح بالضبط.
1,9,corpus-6-test-corpus-26-train,ALE,مالي سمعان من قبل بهالعنوان هون.
2,11,corpus-6-test-corpus-26-train,ALE,روح ساوي لبين ما تشوف صيدلية.
3,26,corpus-6-test-corpus-26-train,ALE,بشقد الفطور؟
4,27,corpus-6-test-corpus-26-train,ALE,شلون بقدر أساعدك؟


# مهمة 02 : فهم وتحليل البيانات

EDA

## [2.1]


## [2.2]

## [2.3]

## [2.4]

## [2.5]

## [2.6]

## [2.7]

# مهمة 03 : تنظيف وتوحيد النص

normalization and cleaning Text

## [3.1]


## [3.2]

## [3.3]

## [3.4]

# مهمة 04  : بناء النموذج المرجعي

Model Baseline

## [4.1]


## [4.2]

## [4.3]

## [4.4]

## [4.5]

## [4.6]

# مهمة 05 : التدريب باستخدام نماذج التعلم العميق

Deep learning

The Comparison Dictionary example

In [ ]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['model_parameters'] = []
comparison_table['preprocessing_methods'] = []
comparison_table['accuracy'] = []
comparison_table['balance_accuracy'] = []

Filling the dictionary example

In [ ]:
comparison_table['question_step_number'].append("3.5")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove links")
comparison_table['accuracy'].append(accuracy)
comparison_table['balance_accuracy'].append(balance_accuracy_value)

## [5.1]

## [5.2]

### [5.2.1]

### [5.2.2]

### [5.2.3]

## [5.3]

## [5.4]

## [5.5]

## [5.6]

# مهمة 06 : التمثيل الداللي

Semantic representation

## [6.1]

## [6.2]

# Final Results

In [ ]:
df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)